In [1]:
from MBDOE import *
from generalize_functions import CEOSModels


[    0.00] Initializing mpi-sppy


## Choose experiments and model

In [2]:
model_option = "PR"
if_poly = False

if model_option=="PR":
    if not if_poly:
        from hfc32_emimtf2n_PR import configuration
    else:
        from hfc32_emimtf2n_PR_polynomial import configuration
        
elif model_option=="SRK":
    if not if_poly:
        from hfc32_emimtf2n_SRK import configuration
    else:
        from hfc32_emimtf2n_SRK_polynomial import configuration
        

In [3]:
# Experiments 
data_file = './emimtf2n/R32/Final_Results/MBDoE/r32_emimtf2n_subset.csv'

path = './emimtf2n/R32/Final_Results/MBDoE/Params/'

# Decide on model 
#para_file = path + 'PR_params_1param_Opt1.csv'
#PR_type_opt = "1Param_Opt1"

#para_file = path + 'PR_params_1param_Opt2.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'PR_params_1param_Opt2_GF.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'PR_params_3params_Opt1.csv'
#PR_type_opt = "3Params_Opt1"

#para_file = path + 'PR_params_3params_Opt2.csv'
#PR_type_opt = "3Params_Opt2"

#para_file = path + 'PR_params_linTdep.csv'
#PR_type_opt = "Linear"

#para_file = path + 'PR_params_noTdep.csv'
#PR_type_opt = "No"

#para_file = path + 'PR_params_polyTdep.csv'
#PR_type_opt = "Polynomial"

# important model
para_file = path + 'PR_params_quadTdep.csv'
PR_type_opt = "Quadratic"

#para_file = path + 'SRK_params_1param_Opt1.csv'
#PR_type_opt = "1Param_Opt1"

#para_file = path + 'SRK_params_1param_Opt2.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'SRK_params_3params_Opt1.csv'
#PR_type_opt = "3Params_Opt1"

#para_file = path + 'SRK_params_3params_Opt2.csv'
#PR_type_opt = "3Params_Opt2"

#para_file = path + 'SRK_params_linTdep.csv'
#PR_type_opt = "Linear"

#para_file = path + 'SRK_params_noTdep.csv'
#PR_type_opt = "No"


#para_file = path + 'SRK_params_polyTdep.csv'
#PR_type_opt = "Polynomial"

#para_file = path + 'SRK_params_quadTdep.csv'
#PR_type_opt = "Quadratic"



model_creation = CEOSModels(para_file, configuration, 
                         comp_1= "R32", comp_2 = "emimTf2N", 
                         x_comp_1="x_R32", x_comp_2="x_emimTf2N",  Model_type = model_option, Tdep_type=PR_type_opt)

## Run pyomo.doe

In [4]:
data_exp = pd.read_csv(data_file)
print(data_exp)
print(data_exp.iloc[1])

    temperature  pressure  x_R32  x_emimTf2N
0        283.15    100100  0.136       0.864
1        283.15    250300  0.306       0.694
2        283.15    399300  0.448       0.552
3        283.15    549300  0.570       0.430
4        283.15    700400  0.672       0.328
5        283.15    849400  0.786       0.214
6        298.15     99800  0.095       0.905
7        298.15    250200  0.218       0.782
8        298.15    399400  0.325       0.675
9        298.15    549400  0.417       0.583
10       298.15    699500  0.499       0.501
11       298.15    849500  0.570       0.430
12       298.15    999700  0.643       0.357
13       323.15    100300  0.055       0.945
14       323.15    249700  0.133       0.867
15       323.15    400100  0.200       0.800
16       323.15    549500  0.264       0.736
17       323.15    700300  0.320       0.680
18       323.15    850400  0.368       0.632
19       323.15   1000400  0.417       0.583
20       348.05     99800  0.034       0.966
21       3

In [ ]:
print(model_creation.param_name_dict)

In [ ]:
mbdoe_obj = MBDOE(data_file, model_creation)

exp_set = range(len(data_exp))
#exp_set = range(12)
#exp_set = [0,1,2,3]

#exp_set = 0
# init_T: 283-348; 323 is good; init_pressure: low mole fraction use 150000; high 399800; x_option: 0.4-0.6;0.3-0.55;
# SRK: init_temp_option=323, init_pressure_option= 399800, init_x_c1_option = 0.5

total_fim = mbdoe_obj.sumDOE(exp_set, scale_opt=True, record_name="SRK_polyTdep", 
                             init_temp_option=343, 
                             init_pressure_option= 150000,
                             init_x_c1_option = 0.4,
                             poly_option=True)
#total_fim = mbdoe_obj.doe(exp_set, scale=True)

In [ ]:
print(total_fim)
print(np.linalg.det(total_fim))
print(np.trace(total_fim))
print(np.linalg.eigvals(total_fim))

## Heatmap

In [5]:
# read prior

f = open('emimtf2n_FIM_info/PR_quadTdep.json')

data = json.load(f)

prior_info = data[0]['Total']
#print(prior_info)

for i in range(6):
    for j in range(6):
        prior_info[i][j] *= 10**(-10)

print(prior_info)
print(np.linalg.det(prior_info))
print(np.log10(np.linalg.det(prior_info)))

[[4557.535530992685, -1079.6186966711327, 8344.529017504288, -798.9512603441533, 3003.549094513206, -529.4333373880846], [-1079.6186966711327, 388.88133653439434, -1901.1816632190887, 279.054860245738, -657.7442271879426, 178.98151313955444], [8344.529017504288, -1901.1816632190887, 15359.201236486178, -1413.4663295880396, 5557.3688924934195, -941.178913439035], [-798.9512603441533, 279.054860245738, -1413.4663295880396, 200.80609121460168, -491.37691190048275, 129.18219737542094], [3003.549094513206, -657.7442271879426, 5557.3688924934195, -491.37691190048275, 2021.1335904074926, -328.8302607352068], [-529.4333373880846, 178.98151313955444, -941.178913439035, 129.18219737542094, -328.8302607352068, 83.37442064565272]]
11.49965617183968
1.0606848555792616


In [ ]:
#data_exp = pd.read_csv(data_file)
#print(data_exp)

In [7]:
# model list 
#T_range = [273, 360]
#x_range = [0.1, 0.9]

T_range = [273, 283, 293, 303, 313, 323, 333, 343, 353, 363, 373, 383, 393, 400]
#x_range = [0.1,0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

x_range = [0.01, 0.05]

data_exp = {'pressure': 549300}

fixed_model_set = []

for i in range(len(T_range)):
    for j in range(len(x_range)):
        data_exp['temperature'] = T_range[i]
        data_exp['x_R32'] = x_range[j]
        data_exp['x_emimTf2N'] = 1-x_range[j]
    
        print(data_exp)
        
        model = model_creation.create_model(data_exp)
        
        print(model.fs.state_block.pressure.value)
        
        fixed_model_set.append(model)

{'pressure': 549300, 'temperature': 273, 'x_R32': 0.01, 'x_emimTf2N': 0.99}
2022-10-21 10:56:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.05, 'x_emimTf2N': 0.95}
2022-10-21 10:56:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 283, 'x_R32': 0.01, 'x_emimTf2N': 0.99}
2022-10-21 10:56:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 283, 'x_R32': 0.05, 'x_emimTf2N': 0.95}
2022-10-21 10:56:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 293, 'x_R32': 0.01, 'x_emimTf2N': 0.99}
2022-10-21 10:56:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Sol

In [8]:
design_range_values = [T_range, x_range]

In [9]:
mbdoe_obj = MBDOE(data_file, model_creation)

all_fim = mbdoe_obj.run_grid_search(design_range_values, fixed_models=fixed_model_set, 
                                    prior_FIM = prior_info, 
                                    scale_opt=True,
                                    scale_pressure=0.00001,
                                   store_name="PR_quad_543900_scaledBar_under10")
                                    #, record_name="SRK_polyTdep", 
                                    # init_temp_option=343, 
                                    # init_pressure_option= 150000,
                                    # init_x_c1_option = 0.4,
                                    # poly_option=True)



parameter set: {'fs.properties.PR_kappa_A["R32", "emimTf2N"]': 0.3212980565927437, 'fs.properties.PR_kappa_A["emimTf2N", "R32"]': 0.8429816989080334, 'fs.properties.PR_kappa_B["R32", "emimTf2N"]': -0.5532108602621684, 'fs.properties.PR_kappa_B["emimTf2N", "R32"]': -0.6100855368792957, 'fs.properties.PR_kappa_C["R32", "emimTf2N"]': 0.1862684932461356, 'fs.properties.PR_kappa_C["emimTf2N", "R32"]': -0.3935451934926509}
All measurements are flattened.
Flatten measurement name: ['fs.state_block.pressure']
28  design vectors will be searched.
=======This is the  1 th iteration=======
Design variable values of this iteration: {'fs.F101.inlet.temperature': {0: 1}, 'fs.F101.inlet.pressure': {0: 1}, 'fs.F101.inlet.mole_frac_comp[0,"R32"]': {0: 1}, "fs.F101.inlet.mole_frac_comp[0,'emimTf2N']": {0: 1}}
Sensitivity information is scaled by its corresponding parameter nominal value.
Sensitivity information is scaled by constant  1e-05  times itself.
After practice: {'fs.state_block.pressure': [0]}


pressure: 9562.502459941017
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 9.059906005859375e-06
Solve time with direct kaug mode [s]: 0.5104222297668457
Total wall clock time [s]: 0.5111222267150879
Existed information has been added.
FIM: [[ 4557.61217738 -1079.61893503  8344.65878473  -798.95141829
   3003.59205848  -529.43343017]
 [-1079.61893503   388.88133728 -1901.18206678   279.05486074
   -657.7443608    178.98151343]
 [ 8344.65878473 -1901.18206678 15359.42094068 -1413.466597
   5557.44163324  -941.17907053]
 [ -798.95141829   279.05486074 -1413.466597     200.80609154
   -491.37700044   129.18219757]
 [ 3003.59205848  -657.7443608   5557.44163324  -491.37700044
   2021.15767377  -328.83031275]
 [ -529.43343017   178.98151343  -941.17907053   129.18219757
   -328.83031275    83.37442076]]
Trace: 22611.252641412502
Determinant: 11.502262257913932
Condition number: 271611307.35909224
Minimal eigen value: 8.224050946408187e-05
Eigen values: [2.23374523e+04 2

pressure: 14636.271525683622
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 7.3909759521484375e-06
Solve time with direct kaug mode [s]: 0.46836233139038086
Total wall clock time [s]: 0.4690430164337158
Existed information has been added.
FIM: [[ 4557.68058308 -1079.61915651  8344.79134914  -798.95158899
   3003.64187217  -529.4335459 ]
 [-1079.61915651   388.88133799 -1901.18249486   279.05486129
   -657.74452131   178.9815138 ]
 [ 8344.79134914 -1901.18249486 15359.67567217 -1413.46692395
   5557.53668405  -941.17929054]
 [ -798.95158899   279.05486129 -1413.46692395   200.80609196
   -491.37712211   129.18219785]
 [ 3003.64187217  -657.74452131  5557.53668405  -491.37712211
   2021.1929325   -328.8303941 ]
 [ -529.4335459    178.9815138   -941.17929054   129.18219785
   -328.8303941     83.37442095]]
Trace: 22611.611038647985
Determinant: 11.50527133286585
Condition number: 271614955.1875701
Minimal eigen value: 8.224070146949959e-05
Eigen values: [2.23378044e+0

pressure: 21376.68124174275
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 1.0967254638671875e-05
Solve time with direct kaug mode [s]: 0.5064349174499512
Total wall clock time [s]: 0.5086524486541748
Existed information has been added.
FIM: [[ 4557.78767532 -1079.61951256  8345.01414459  -798.95188568
   3003.73162185  -529.43376331]
 [-1079.61951256   388.88133917 -1901.18323299   279.05486227
   -657.74481781   178.98151452]
 [ 8345.01414459 -1901.18323299 15360.13462368 -1413.46753274
   5557.7200761   -941.17973291]
 [ -798.95188568   279.05486227 -1413.46753274   200.80609277
   -491.37736458   129.18219843]
 [ 3003.73162185  -657.74481781  5557.7200761   -491.37736458
   2021.26572199  -328.83056906]
 [ -529.43376331   178.98151452  -941.17973291   129.18219843
   -328.83056906    83.37442137]]
Trace: 22612.24987429297
Determinant: 11.513771859211365
Condition number: 271588917.13934296
Minimal eigen value: 8.225089125197652e-05
Eigen values: [2.23384305e+04

pressure: 29997.972232447566
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 8.344650268554688e-06
Solve time with direct kaug mode [s]: 0.5107300281524658
Total wall clock time [s]: 0.5119335651397705
Existed information has been added.
FIM: [[ 4557.9431959  -1079.62004497  8345.36044106  -798.95236327
   3003.88068908  -529.43413971]
 [-1079.62004497   388.88134099 -1901.18441305   279.05486389
   -657.74532389   178.98151579]
 [ 8345.36044106 -1901.18441305 15360.89690644 -1413.46857897
   5558.04517228  -941.18054975]
 [ -798.95236327   279.05486389 -1413.46857897   200.8060942
   -491.37780902   129.18219955]
 [ 3003.88068908  -657.74532389  5558.04517228  -491.37780902
   2021.40330938  -328.83091334]
 [ -529.43413971   178.98151579  -941.18054975   129.18219955
   -328.83091334    83.37442222]]
Trace: 22613.305269135748
Determinant: 11.55388343224577
Condition number: 270808737.91062385
Minimal eigen value: 8.249165018059954e-05
Eigen values: [2.23394597e+04 

pressure: 40666.54937313892
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 7.62939453125e-06
Solve time with direct kaug mode [s]: 0.5100750923156738
Total wall clock time [s]: 0.5106899738311768
Existed information has been added.
FIM: [[ 4558.15461802 -1079.62079272  8345.86311947  -798.95308266
   3004.11129658  -529.4347471 ]
 [-1079.62079272   388.88134363 -1901.1861801    279.05486642
   -657.74613064   178.98151791]
 [ 8345.86311947 -1901.1861801  15362.07616035 -1413.4702566
   5558.58041018  -941.18195129]
 [ -798.95308266   279.05486642 -1413.4702566    200.80609658
   -491.37856678   129.18220152]
 [ 3004.11129658  -657.74613064  5558.58041018  -491.37856678
   2021.6441344   -328.83154091]
 [ -529.4347471    178.98151791  -941.18195129   129.18220152
   -328.83154091    83.37442386]]
Trace: 22614.93677683121
Determinant: 11.799534922996203
Condition number: 266309464.6934803
Minimal eigen value: 8.389127206188826e-05
Eigen values: [2.23410398e+04 2.6043

pressure: 53487.83817129731
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 9.059906005859375e-06
Solve time with direct kaug mode [s]: 0.5073351860046387
Total wall clock time [s]: 0.5080525875091553
Existed information has been added.
FIM: [[ 4558.42551317 -1079.62178641  8346.54965386  -798.95410503
   3004.44623634  -529.43566873]
 [-1079.62178641   388.88134726 -1901.18867824   279.05487012
   -657.74734179   178.98152123]
 [ 8346.54965386 -1901.18867824 15363.78893653 -1413.47278822
   5559.40578428  -941.18420656]
 [ -798.95410503   279.05487012 -1413.47278822   200.80610031
   -491.37977947   129.18220483]
 [ 3004.44623634  -657.74734179  5559.40578428  -491.37977947
   2022.03794948  -328.83261083]
 [ -529.43566873   178.98152123  -941.18420656   129.18220483
   -328.83261083    83.37442675]]
Trace: 22617.314273496297
Determinant: 12.827070079076137
Condition number: 251543014.1743288
Minimal eigen value: 8.882505704988481e-05
Eigen values: [2.23433226e+04 

In [ ]:
fixed = {}

all_fim.extract_criteria()
print(all_fim.store_all_results_dataframe)

all_fim.figure_drawing(fixed, ['Temperature','x_R32'], 'PR quadratic','Temperature [K]', 'x (R32)' )
